In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D

import keras_tuner

#import custom modules
import sys
sys.path.insert(0, '/Users/frederickkorbel/Documents/projects/paper/mlcis/utils')

import integrated_gradients as ig
import metaplot

#set seaborn style
sns.set_style('ticks')

In [ ]:
def test_data(df, model, test_seq, obs_col, output_col='pred'):
    '''Predict mean ribosome load using model and test set UTRs'''
    
    # Scale the test set mean ribosome load
    scaler = preprocessing.StandardScaler()
    scaler.fit(df[obs_col].values.reshape(-1,1))
    
    # Make predictions
    predictions = model.predict(test_seq).reshape(-1,1)
    
    # Inverse scaled predicted mean ribosome load and return in a column labeled 'pred'
    df.loc[:,output_col] = scaler.inverse_transform(predictions)
    return df


def r2(x,y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return r_value**2

<font color=white size=20> Defining Datasets</font>

In [ ]:
#defining the two datasets for subsequent training
train_synthetic=pd.read_csv('/Users/frederickkorbel/Documents/projects/paper/mlcis/data/random_train.csv.gz', compression='gzip', index_col=[0])
train_human=pd.read_csv('/Users/frederickkorbel/Documents/projects/paper/mlcis/data/human_train.csv', index_col=[0])

#the dataset to optimize for and test performance on
test_human=pd.read_csv('/Users/frederickkorbel/Documents/projects/paper/mlcis/data/human_test.csv', index_col=[0])

#one-hot-encode all datasets
train_synthetic_seq=metaplot.one_hot_encode(train_synthetic)
train_human_seq=metaplot.one_hot_encode(train_human)

test_human_seq=metaplot.one_hot_encode(test_human)

#scale MRL values
test_human.loc[:,'scaled_rl'] = preprocessing.StandardScaler().fit_transform(test_human.loc[:,'rl'].values.reshape(-1,1))
train_human.loc[:,'scaled_rl'] = preprocessing.StandardScaler().fit_transform(train_human.loc[:,'rl'].values.reshape(-1,1))


In [ ]:
#define the hyperparameters to optimize via grid search
epochs_hum=[1,3,5,7,9]
epochs_syn=[1,2,3]

#perform the grid search to find the optimal number of training epochs

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
history = LossHistory()



class OptimizedModel(keras_tuner.HyperModel):

    def build(self, hp):

        model = Sequential()

        model.add(Conv1D(activation="relu", input_shape=(50, 4), padding='same', filters=120, kernel_size=8))
        model.add(Conv1D(activation="relu", input_shape=(50, 1), padding='same', filters=120, kernel_size=8))
        model.add(Conv1D(activation="relu", input_shape=(50, 1), padding='same', filters=120, kernel_size=8))
        model.add(Flatten())

        model.add(Dense(40))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
    
        model.add(Dense(1))
        model.add(Activation('linear'))

        adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        model.compile(loss='mean_squared_error', optimizer=adam)

        return model


    def fit(self, hp, model, *args, **kwargs):
        

        model.fit(*args, batch_size=128, epochs=hp.Choice('epochs', epochs_hum), **kwargs)

        return model #.fit(*args, batch_size=128, epochs=hp.Choice('epochs', epochs_hum), **kwargs)

tuner=keras_tuner.RandomSearch(
    hypermodel=OptimizedModel(),
    objective='val_loss',
    max_trials=10,
    overwrite=True,
    directory='/Users/frederickkorbel/Documents/projects/paper/mlcis/models/tune_mrl_model',
    project_name='tune_mrl_model'
)

tuner.search(train_human_seq,train_human['rl'], validation_data=(test_human_seq, test_human['rl']))

In [ ]:
epochs_hum=[1,2,3,4,5,6,7,8,9,10]

class OptimizedModel(keras_tuner.HyperModel):

    def build(self, hp):
        return keras.models.load_model('/Users/frederickkorbel/Documents/projects/paper/mlcis/models/main_MRL_model.hdf5')

    def fit(self, hp, model, *args, **kwargs):
        #model.fit(*args, batch_size=128, epochs=hp.Choice('epochs', epochs_syn), **kwargs)
        return model.fit(*args, batch_size=128, epochs=hp.Choice('epochs', epochs_hum), **kwargs)


tuner=keras_tuner.RandomSearch(
    hypermodel=OptimizedModel(),
    objective='val_loss',
    max_trials=10,
    overwrite=True,
    directory='/Users/frederickkorbel/Documents/projects/paper/mlcis/models/tune_mrl_model',
    project_name='tune_mrl_model')

tuner.search(train_human_seq,train_human['rl'], validation_data=(test_human_seq, test_human['rl']))

tuner.results_summary(num_trials=10)

best_model = tuner.get_best_models()[0]

In [ ]:
#plot r2 of all generated models as heatmap


In [ ]:
#performance of the optimized model
OPT_human_test_pred=test_data(test_human, best_model, 'rl', test_human_seq)

r_2 = r2(test_human['rl'], OPT_human_test_pred['pred'])
print('r-squared = ', r_2)
g, ax = plt.subplots(figsize=(6,6))
g = sns.regplot(data = OPT_human_test_pred, x = 'rl', y = 'pred', scatter_kws={'alpha':0.5}, line_kws={"color": "black"})
g.set(ylim=(0,9), xlim=(0,9), xticks = range(1,10,1), yticks = range(1,10,1))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Measured MRL', fontsize=20)
plt.ylabel('Predicted MRL', fontsize=20)
sns.despine()

#save predictions
#OPT_human_test_pred.to_csv()

In [ ]:
#compare performance of best model to O5P (original MRL model) and hMRL (model trained on human 5'UTRs)

mrl_model=
hmrl_model=
